In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import markers,lines,colors
from scipy import interpolate
import ht,fluids
import CoolProp as cp
import pandas as pd
from IPython.display import  Latex

In [8]:
texto = "En un intercambiador de calor del tipo tubos concéntricos, se enfría aceite con agua. El aceite circula por un tubo interno de latón cuyo diámetro es \(d_2/d_1=14/12\)mm a una velocidad \(v_1=4\)m/s. La temperatura del aceite a la entrada en el intercambiador de calor es \(t_1'=100^\circ\)C. El agua se mueve en contracorriente respecto al aceite por la holgura anular a una velocidad \(v_2=2.5\)m/s y su temperatura a la entrada es \(t_2'=20^\circ\)C. El diámetro interno del tubo exterior es \(d_3=22\)mm."
texto2 = "Determinar la longitud total de la superficie de intercambio de calor con la cual la temperatura del aceite a la salida será \(t_1''=60^\circ\)C. Se pueden despreciar las pérdidas de calor a través de la superficie externa del intercambiado"
display(Latex(texto))
display(Latex(texto2))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [9]:

t1e = 100
t2e = 20
v_1 = 4.
v_2 = 2.5
d1,d2,d3 = np.array([12,14,22])*1e-3
espesor = (d2-d1)/2
t1s = 60

t1m = (t1e+t1s)/2

#MK1
cp_1 = 1989
k_1 = 0.134
rho_1 = 856
mu_1 = 400e-4
nu_1 = mu_1/rho_1
Pr_1 = nu_1/(k_1/(rho_1*cp_1))
#transformadores
cp_1 = 2026
k_1 = 0.1056
rho_1 = 843
mu_1 = 30.8e-4
nu_1 = mu_1/rho_1
Pr_1 = nu_1/(k_1/(rho_1*cp_1))


cp_1 = cp.CoolProp.PropsSI('C','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
rho_1 = cp.CoolProp.PropsSI('D','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
mu_1 = cp.CoolProp.PropsSI('V','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
nu_1 = mu_1/rho_1
Pr_1 = cp.CoolProp.PropsSI('Prandtl','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
k_1 = cp.CoolProp.PropsSI('L','T',273+t1m,'P',101.3e3,'INCOMP::TCO')

print(rho_1)


cp_2 = cp.CoolProp.PropsSI('C','T',273+t1m,'P',101.3e3,'Water')
rho_2 = cp.CoolProp.PropsSI('D','T',273+t1m,'P',101.3e3,'Water')
mu_2 = cp.CoolProp.PropsSI('V','T',273+t1m,'P',101.3e3,'Water')
nu_2 = mu_2/rho_2
Pr_2 = cp.CoolProp.PropsSI('Prandtl','T',273+t1m,'P',101.3e3,'Water')
k_2 = cp.CoolProp.PropsSI('L','T',273+t1m,'P',101.3e3,'Water')
area_1 = np.pi*d1**2/4
caudal_1 = area_1 *v_1
area_2 = np.pi*d3**2/4-np.pi*d2**2/4
caudal_2 = area_2 * v_2
C_1 = caudal_1*rho_1*cp_1
C_2 = caudal_2*rho_2*cp_2
Cmin = np.min((C_1,C_2))
Cmax = np.max((C_1,C_2))
Q_1 = C_1 * (t1e-t1s)

Q_2 = Q_1
t2s = Q_2/C_2+t2e
Qmax = Cmin*(t1e-t2e)
nmin = np.nonzero(np.array([C_1,C_2])==Cmin)[0]+1
nmax = np.nonzero(np.array([C_1,C_2])==Cmax)[0]+1
Cr = Cmin/Cmax 

796.3274019999999


In [10]:
cp_1 = cp.CoolProp.PropsSI('C','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
rho_1 = cp.CoolProp.PropsSI('D','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
mu_1 = cp.CoolProp.PropsSI('V','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
nu_1 = mu_1/rho_1
Pr_1 = cp.CoolProp.PropsSI('Prandtl','T',273+t1m,'P',101.3e3,'INCOMP::TCO')
k_1 = cp.CoolProp.PropsSI('L','T',273+t1m,'P',101.3e3,'INCOMP::TCO')

In [11]:
cp_1,rho_1,mu_1,nu_1,k_1

(2064.9877,
 796.3274019999999,
 0.0008122171103526035,
 1.0199537380136564e-06,
 0.108832644)

In [12]:
display(Latex(f"Considere los siguientes valores para las propiedades del aceite:"))
display(Latex(f"\(Cp_1\) = {cp_1:.1f} J/K kg\(~~~~\)\(k_1\) = {k_1:.4f} W/m"))
display(Latex(f"\(\rho_1 = {rho_1:.1f} \)kg/m\(^3~~~~\)\(\mu_1 = {mu_1*1e4:.2f}\cdot10^{{-4}} \)kg/m s"))

#mu_1 = 30.8e-4
nu_1 = mu_1/rho_1
Pr_1 = nu_1/(k_1/(rho_1*cp_1))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [13]:
Pr_1

15.410985904262962

In [14]:
k_1/rho_1/cp_1,mu_1/rho_1

(6.618354882353883e-08, 1.0199537380136564e-06)

In [15]:
print('Calor intercambiado: Q=%.2f kW'%(Q_1*1e-3))
print('Calor maximo: Q=%.2f kW'%(Qmax*1e-3))
print('Temperatura salida agua %.1f C'%t2s)
print('C_R = %.2f'%Cr)
print('Fluido %0d Cmin'%nmin[0])


Calor intercambiado: Q=29.76 kW
Calor maximo: Q=59.51 kW
Temperatura salida agua 32.9 C
C_R = 0.32
Fluido 1 Cmin


In [16]:
effectiveness = Q_1/Qmax
Nut = ht.NTU_from_effectiveness(effectiveness, Cr, subtype='counterflow')
UA = Nut*Cmin
print('Eficiencia = %.2f'%effectiveness)
print('Nut = %.2f'%Nut)
print('UA = %.1f'%UA)

Eficiencia = 0.50
Nut = 0.76
UA = 568.0


In [17]:
Re_1 = d1*v_1/mu_1*rho_1
perimetro_2 = np.pi*d2+np.pi*d3
dhidr = area_2/perimetro_2
Re_2 = dhidr*v_1/mu_2*rho_2
Nu_1 = ht.conv_internal.laminar_T_const()
Nu_1 = ht.conv_internal.turbulent_Colburn(Re_1,Pr_1)
Nu_2 = ht.conv_internal.turbulent_Colburn(Re_2,Pr_2)
print('Reynolds 1 = %0d'%Re_1)
print('Reynolds 2 = %0d'%Re_2)
print('Nusselt 1 = %0d'%Nu_1)
print('Nusselt 2 = %0d'%Nu_2)
hc_1 = Nu_1*k_1/d1 
hc_2 = Nu_2*k_2/dhidr 
print('hc 1 = %0d'%hc_1)
print('hc 2 = %0d'%hc_2)

Reynolds 1 = 47060
Reynolds 2 = 21919
Nusselt 1 = 313
Nusselt 2 = 89
hc 1 = 2840
hc 2 = 29761


In [18]:
laton = ht.insulation.nearest_material('brass')
k_laton = ht.k_material(laton)

Rtotal = 1/(hc_1)+d1/(hc_2*dhidr)+espesor/k_laton
U = 1/Rtotal

Atotal = UA/U
L = Atotal/(np.pi*d2)
print('U = %.1f W/m2'%U)
print('Area total = %.3f m2'%Atotal)
print('Longitud total L=%.2f m'%(L))

U = 1779.4 W/m2
Area total = 0.319 m2
Longitud total L=7.26 m


In [19]:
ht.conv_internal.turbulent_Colburn(9000,10)

72.18636562688543

In [20]:
#Se reduce el caudal de agua un 20%, como cambian las temperaturas de salida?
C_2_ = C_2*0.8

Cr2 = Cmin/C_2

t2s_ = Q_2/C_2_+t2e
t1s_ = t1e-Q_2/C_1

print(t2s_,t1s_)

36.1269780579398 60.0


In [21]:
#Temp aceite ingreso es 120
t1e_ = 120

Qmax_ = C_1*(t1e_-t2e)
Q_2_ = effectiveness*Qmax_

t2s__ = Q_2_/C_2+t2e
t1s__ = t1e_-Q_2_/C_1

print(t2s__,t1s__)

36.12697805793981 70.0


In [22]:
print(t2s,t1s)

32.90158244635184 60
